In [1]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Load tickerdata
tickerdata = pd.read_excel(r'Data for Thesis.xlsx',engine='openpyxl')
tickerdata['Ticker'] = tickerdata['Ticker'].astype(str)
tickerdata.head()

,Sharing Economy Firm,t,Incumbent Firms,Ticker,Exchange,CI,t-1,t+1,t-3,t+3,...,t+12,ISP (t),ISP (t-1),ISP (t+1),ISP (t-3),ISP (t+3),ISP (t-6),ISP (t+6),ISP (t-12),ISP (t+12)
0,Airbnb,2008-08-31,Marriott International,MAR,NASDAQ,1,2008-07-31,2008-09-30,2008-05-31,2008-11-30,...,2009-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Airbnb,2008-08-31,Las Vegas Sands,LVS,NYSE,0,2008-07-31,2008-09-30,2008-05-31,2008-11-30,...,2009-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bird,2017-09-30,Giant Manufacturing Co Ltd,9921,TPE,1,2017-08-30,2017-10-30,2017-06-30,2017-12-30,...,2018-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bird,2017-09-30,Fox Factory Holding Corp,FOXF,NASDAQ,0,2017-08-30,2017-10-30,2017-06-30,2017-12-30,...,2018-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Boatsetter,2012-10-31,Brunswick Corp,BC,NYSE,1,2012-09-30,2012-11-30,2012-07-31,2013-01-31,...,2013-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#View names of tickerdata.columns
tickerdata.columns

Index(['Sharing Economy Firm', 't', 'Incumbent Firms', 'Ticker', 'Exchange',
       'CI', 't-1', 't+1', 't-3', 't+3', 't-6', 't+6', 't-12', 't+12',
       'ISP (t)', 'ISP (t-1)', 'ISP (t+1)', 'ISP (t-3)', 'ISP (t+3)',
       'ISP (t-6)', 'ISP (t+6)', 'ISP (t-12)', 'ISP (t+12)'],
      dtype='object')

In [4]:
#Reshape/melt ticker data to make time an attribute
transformedtickerdata = tickerdata.melt(id_vars=['Sharing Economy Firm', 'Incumbent Firms', 'Ticker',
       'Exchange', 'CI', 'ISP (t)', 'ISP (t-1)', 'ISP (t+1)', 'ISP (t-3)', 'ISP (t+3)',
       'ISP (t-6)', 'ISP (t+6)', 'ISP (t-12)', 'ISP (t+12)'],var_name='t',value_name='Date').drop(['ISP (t-1)', 'ISP (t+1)', 'ISP (t-3)', 'ISP (t+3)',
       'ISP (t-6)', 'ISP (t+6)', 'ISP (t-12)', 'ISP (t+12)'],1)
transformedtickerdata['Ticker'] = transformedtickerdata['Ticker'].astype(str)
transformedtickerdata.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,ISP (t),t,Date
0,Airbnb,Marriott International,MAR,NASDAQ,1,NaN,t,2008-08-31
1,Airbnb,Las Vegas Sands,LVS,NYSE,0,NaN,t,2008-08-31
2,Bird,Giant Manufacturing Co Ltd,9921,TPE,1,NaN,t,2017-09-30
3,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,NaN,t,2017-09-30
4,Boatsetter,Brunswick Corp,BC,NYSE,1,NaN,t,2012-10-31


In [5]:
#Read price data and transform
pricedata = pd.read_excel(r'Bloomberg Stock Price Data.xlsx', engine='openpyxl',usecols=[0,1],sheet_name=None)
datalist = []
for ticker, data in pricedata.items():
    data['Ticker'] = str(ticker)
    datalist.append(data)
transformedpricedata = pd.concat(datalist,axis=0)
transformedpricedata.dropna(axis=1,how='all')

,Date,Last Price,Ticker
0,2009-08-31,22.4513,MAR
1,2009-08-30,23.0337,MAR
2,2009-08-29,23.0337,MAR
3,2009-08-28,23.0337,MAR
4,2009-08-27,22.9022,MAR
...,...,...,...
748,2005-03-13,38.2311,TRI
749,2005-03-12,38.2311,TRI
750,2005-03-11,38.2311,TRI
751,2005-03-10,38.3192,TRI


In [6]:
#Merge datasets
merged_data = transformedtickerdata.merge(transformedpricedata, how='left',left_on=['Date','Ticker'],right_on=['Date','Ticker']).dropna(axis=1,how='all').rename(columns={'Last Price':'ISP (t)'})
complete_data = merged_data.pivot(index=['Sharing Economy Firm','Incumbent Firms','Ticker','Exchange','CI'],columns=['t'])
complete_data.head()

Date  \
t                                                                           t   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2008-08-31   
                     Marriott International     MAR    NASDAQ   1  2008-08-31   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2017-09-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2017-09-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2012-10-31   

                                                                               \
t                                                                         t+1   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2008-09-30   
                     Marriott International     MAR    NASDAQ   1  2008-09-30   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2017-10-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2017-10-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2012-11-30   

                                                                               \
t                                                                        t+12   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2009-08-31   
                     Marriott International     MAR    NASDAQ   1  2009-08-31   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2018-09-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2018-09-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2013-10-31   

                                                                               \
t                                                                         t+3   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2008-11-30   
                     Marriott International     MAR    NASDAQ   1  2008-11-30   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2017-12-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2017-12-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2013-01-31   

                                                                               \
t                                                                         t+6   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2009-02-28   
                     Marriott International     MAR    NASDAQ   1  2009-02-28   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2018-03-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2018-03-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2013-04-30   

                                                                               \
t                                                                         t-1   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2008-07-31   
                     Marriott International     MAR    NASDAQ   1  2008-07-31   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2017-08-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2017-08-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2012-09-30   

                                                                               \
t                                                                        t-12   
Sharing Economy F

In [7]:
#Drop Multindex
complete_data.columns = ['_'.join(col) for col in complete_data.columns]
complete_data.head()

Date_t  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2008-08-31   
                     Marriott International     MAR    NASDAQ   1  2008-08-31   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2017-09-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2017-09-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2012-10-31   

                                                                     Date_t+1  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2008-09-30   
                     Marriott International     MAR    NASDAQ   1  2008-09-30   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2017-10-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2017-10-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2012-11-30   

                                                                    Date_t+12  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2009-08-31   
                     Marriott International     MAR    NASDAQ   1  2009-08-31   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2018-09-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2018-09-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2013-10-31   

                                                                     Date_t+3  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2008-11-30   
                     Marriott International     MAR    NASDAQ   1  2008-11-30   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2017-12-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2017-12-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2013-01-31   

                                                                     Date_t+6  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2009-02-28   
                     Marriott International     MAR    NASDAQ   1  2009-02-28   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2018-03-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2018-03-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2013-04-30   

                                                                     Date_t-1  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2008-07-31   
                     Marriott International     MAR    NASDAQ   1  2008-07-31   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2017-08-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2017-08-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2012-09-30   

                                                                    Date_t-12  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI              
Airbnb               Las Vegas Sands            LVS    NYSE     0  2007-08-31   
                     Marriott International     MAR    NASDAQ   1  2007-08-31   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  2016-09-30   
                     Giant Manufacturing Co Ltd 9921   TPE      1  2016-09-30   
Boatsetter           Brunswick Corp             BC     NYSE     1  2011-10-31   

                                                                     Date_t-3  \
Sharing Econom

In [8]:
#Delete Date Columns/Reset Index
complete_data = complete_data.drop(columns=['Date_t', 'Date_t+1','Date_t+12','Date_t+3','Date_t+6','Date_t-1','Date_t-12','Date_t-3','Date_t-6']).reset_index().rename(columns={'ISP (t)_t':'ISP(t)','ISP (t)_t+1':'ISP(t+1)','ISP (t)_t+12':'ISP(t+12)','ISP (t)_t+3':'ISP(t+3)','ISP (t)_t+6':'ISP(t+6)','ISP (t)_t-1':'ISP(t-1)','ISP (t)_t-12':'ISP(t-12)','ISP (t)_t-3':'ISP(t-3)','ISP (t)_t-6':'ISP(t-6)'})
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,ISP(t),ISP(t+1),ISP(t+12),ISP(t+3),ISP(t+6),ISP(t-1),ISP(t-12),ISP(t-3),ISP(t-6)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,47.4100,36.1100,14.2600,5.1700,2.2800,45.5200,99.7000,69.4400,83.3000
1,Airbnb,Marriott International,MAR,NASDAQ,1,26.3029,24.3262,22.4513,15.6549,13.2027,24.1584,41.4171,30.6852,31.7947
2,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,43.1000,42.9500,70.0500,38.8500,34.9000,39.8000,22.9700,35.6000,28.6000
3,Bird,Giant Manufacturing Co Ltd,9921,TPE,1,143.0000,154.0000,131.0000,163.5000,154.0000,153.0000,221.0000,174.0000,179.0000
4,Boatsetter,Brunswick Corp,BC,NYSE,1,23.5900,25.7700,45.1300,36.1600,31.6600,22.6300,17.6600,21.9900,26.2900


In [9]:
#Add Columns for Growth Rate % for t=((+-)1,3,6,12)
complete_data['% Change in Share Price (t-1)'] = ((complete_data['ISP(t)']-complete_data['ISP(t-1)'])/complete_data['ISP(t-1)'])*100
complete_data['% Change in Share Price (t+1)'] = ((complete_data['ISP(t+1)']-complete_data['ISP(t)'])/complete_data['ISP(t)'])*100
complete_data['% Change in Share Price (t-3)'] = ((complete_data['ISP(t)']-complete_data['ISP(t-3)'])/complete_data['ISP(t-3)'])*100
complete_data['% Change in Share Price (t+3)'] = ((complete_data['ISP(t+3)']-complete_data['ISP(t)'])/complete_data['ISP(t)'])*100
complete_data['% Change in Share Price (t-6)'] = ((complete_data['ISP(t)']-complete_data['ISP(t-6)'])/complete_data['ISP(t-6)'])*100
complete_data['% Change in Share Price (t+6)'] = ((complete_data['ISP(t+6)']-complete_data['ISP(t)'])/complete_data['ISP(t)'])*100
complete_data['% Change in Share Price (t-12)'] = ((complete_data['ISP(t)']-complete_data['ISP(t-12)'])/complete_data['ISP(t-12)'])*100
complete_data['% Change in Share Price (t+12)'] = ((complete_data['ISP(t+12)']-complete_data['ISP(t)'])/complete_data['ISP(t)'])*100

In [10]:
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,ISP(t),ISP(t+1),ISP(t+12),ISP(t+3),ISP(t+6),...,ISP(t-3),ISP(t-6),% Change in Share Price (t-1),% Change in Share Price (t+1),% Change in Share Price (t-3),% Change in Share Price (t+3),% Change in Share Price (t-6),% Change in Share Price (t+6),% Change in Share Price (t-12),% Change in Share Price (t+12)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,47.4100,36.1100,14.2600,5.1700,2.2800,...,69.4400,83.3000,4.152021,-23.834634,-31.725230,-89.095128,-43.085234,-95.190888,-52.447342,-69.921957
1,Airbnb,Marriott International,MAR,NASDAQ,1,26.3029,24.3262,22.4513,15.6549,13.2027,...,30.6852,31.7947,8.876830,-7.515141,-14.281478,-40.482228,-17.272690,-49.805155,-36.492656,-14.643252
2,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,43.1000,42.9500,70.0500,38.8500,34.9000,...,35.6000,28.6000,8.291457,-0.348028,21.067416,-9.860789,50.699301,-19.025522,87.636047,62.529002
3,Bird,Giant Manufacturing Co Ltd,9921,TPE,1,143.0000,154.0000,131.0000,163.5000,154.0000,...,174.0000,179.0000,-6.535948,7.692308,-17.816092,14.335664,-20.111732,7.692308,-35.294118,-8.391608
4,Boatsetter,Brunswick Corp,BC,NYSE,1,23.5900,25.7700,45.1300,36.1600,31.6600,...,21.9900,26.2900,4.242156,9.241204,7.276035,53.285290,-10.270065,34.209411,33.578709,91.309877


In [11]:
#Add Columns for % Change in Growth Rate
complete_data['% Change in Growth Rate (t=1)'] = ((complete_data['% Change in Share Price (t+1)']-complete_data['% Change in Share Price (t-1)'])/complete_data['% Change in Share Price (t-1)'])*100
complete_data['% Change in Growth Rate (t=3)'] = ((complete_data['% Change in Share Price (t+3)']-complete_data['% Change in Share Price (t-3)'])/complete_data['% Change in Share Price (t-3)'])*100
complete_data['% Change in Growth Rate (t=6)'] = ((complete_data['% Change in Share Price (t+6)']-complete_data['% Change in Share Price (t-6)'])/complete_data['% Change in Share Price (t-6)'])*100
complete_data['% Change in Growth Rate (t=12)'] = ((complete_data['% Change in Share Price (t+12)']-complete_data['% Change in Share Price (t-12)'])/complete_data['% Change in Share Price (t-12)'])*100
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,ISP(t),ISP(t+1),ISP(t+12),ISP(t+3),ISP(t+6),...,% Change in Share Price (t-3),% Change in Share Price (t+3),% Change in Share Price (t-6),% Change in Share Price (t+6),% Change in Share Price (t-12),% Change in Share Price (t+12),% Change in Growth Rate (t=1),% Change in Growth Rate (t=3),% Change in Growth Rate (t=6),% Change in Growth Rate (t=12)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,47.4100,36.1100,14.2600,5.1700,2.2800,...,-31.725230,-89.095128,-43.085234,-95.190888,-52.447342,-69.921957,-674.048964,180.833666,120.936221,33.318400
1,Airbnb,Marriott International,MAR,NASDAQ,1,26.3029,24.3262,22.4513,15.6549,13.2027,...,-14.281478,-40.482228,-17.272690,-49.805155,-36.492656,-14.643252,-184.660191,183.459660,188.346252,-59.873427
2,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,43.1000,42.9500,70.0500,38.8500,34.9000,...,21.067416,-9.860789,50.699301,-19.025522,87.636047,62.529002,-104.197427,-146.805878,-137.526202,-28.649221
3,Bird,Giant Manufacturing Co Ltd,9921,TPE,1,143.0000,154.0000,131.0000,163.5000,154.0000,...,-17.816092,14.335664,-20.111732,7.692308,-35.294118,-8.391608,-217.692308,-180.464697,-138.247863,-76.223776
4,Boatsetter,Brunswick Corp,BC,NYSE,1,23.5900,25.7700,45.1300,36.1600,31.6600,...,7.276035,53.285290,-10.270065,34.209411,33.578709,91.309877,117.842129,632.339710,-433.098300,171.927897


In [12]:
#Convert Exchange to Dummy Variable
exchange_dummies = pd.get_dummies(complete_data['Exchange'])
complete_data = pd.concat([complete_data,exchange_dummies],axis=1)
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,ISP(t),ISP(t+1),ISP(t+12),ISP(t+3),ISP(t+6),...,% Change in Growth Rate (t=3),% Change in Growth Rate (t=6),% Change in Growth Rate (t=12),NASDAQ,NSE,NYSE,OTCMKTS,TGT,TPE,WMT
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,47.4100,36.1100,14.2600,5.1700,2.2800,...,180.833666,120.936221,33.318400,0,0,1,0,0,0,0
1,Airbnb,Marriott International,MAR,NASDAQ,1,26.3029,24.3262,22.4513,15.6549,13.2027,...,183.459660,188.346252,-59.873427,1,0,0,0,0,0,0
2,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,43.1000,42.9500,70.0500,38.8500,34.9000,...,-146.805878,-137.526202,-28.649221,1,0,0,0,0,0,0
3,Bird,Giant Manufacturing Co Ltd,9921,TPE,1,143.0000,154.0000,131.0000,163.5000,154.0000,...,-180.464697,-138.247863,-76.223776,0,0,0,0,0,1,0
4,Boatsetter,Brunswick Corp,BC,NYSE,1,23.5900,25.7700,45.1300,36.1600,31.6600,...,632.339710,-433.098300,171.927897,0,0,1,0,0,0,0


In [13]:
#Write complete data to Excel
with pd.ExcelWriter('complete_data.xlsx',engine='openpyxl') as writer:
    complete_data.to_excel(writer)

In [14]:
#Columns to Drop for Stata
complete_data.columns

Index(['Sharing Economy Firm', 'Incumbent Firms', 'Ticker', 'Exchange', 'CI',
       'ISP(t)', 'ISP(t+1)', 'ISP(t+12)', 'ISP(t+3)', 'ISP(t+6)', 'ISP(t-1)',
       'ISP(t-12)', 'ISP(t-3)', 'ISP(t-6)', '% Change in Share Price (t-1)',
       '% Change in Share Price (t+1)', '% Change in Share Price (t-3)',
       '% Change in Share Price (t+3)', '% Change in Share Price (t-6)',
       '% Change in Share Price (t+6)', '% Change in Share Price (t-12)',
       '% Change in Share Price (t+12)', '% Change in Growth Rate (t=1)',
       '% Change in Growth Rate (t=3)', '% Change in Growth Rate (t=6)',
       '% Change in Growth Rate (t=12)', 'NASDAQ', 'NSE', 'NYSE', 'OTCMKTS',
       'TGT', 'TPE', 'WMT'],
      dtype='object')

In [15]:
#Create STATA ready data
stata_data = complete_data[['CI', 'NASDAQ', 'NSE', 'NYSE', 'OTCMKTS','TGT', 'TPE', 'WMT', '% Change in Growth Rate (t=1)', '% Change in Growth Rate (t=3)', '% Change in Growth Rate (t=6)','% Change in Growth Rate (t=12)']].rename(columns={'% Change in Growth Rate (t=1)':'one_m_diff','% Change in Growth Rate (t=3)':'three_m_diff','% Change in Growth Rate (t=6)':'six_m_diff','% Change in Growth Rate (t=12)':'one_y_diff'})
stata_data.head()

,CI,NASDAQ,NSE,NYSE,OTCMKTS,TGT,TPE,WMT,one_m_diff,three_m_diff,six_m_diff,one_y_diff
0,0,0,0,1,0,0,0,0,-674.048964,180.833666,120.936221,33.318400
1,1,1,0,0,0,0,0,0,-184.660191,183.459660,188.346252,-59.873427
2,0,1,0,0,0,0,0,0,-104.197427,-146.805878,-137.526202,-28.649221
3,1,0,0,0,0,0,1,0,-217.692308,-180.464697,-138.247863,-76.223776
4,1,0,0,1,0,0,0,0,117.842129,632.339710,-433.098300,171.927897


In [16]:
#Write STATA data to Excel
with pd.ExcelWriter('stata_data.xlsx',engine='openpyxl') as writer:
    stata_data.to_excel(writer)